In [1]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise import dump

train_data= pd.read_csv(r"./testdata.csv")
df=pd.DataFrame(train_data)

df_table1 = df.set_index(["user", "item"])[["rate"]].unstack()
df_table1

#df_table2 = df.set_index(["user", "item"])[["id"]].unstack()
#df_table2

rate                                                        ...         \
item 10005 10006 10014 10025 10029 10040 10051 10097 10137 10166  ...  50002   
user                                                              ...          
1     10.0   1.0   3.0   1.0   NaN   1.0   NaN   NaN   NaN   1.0  ...    NaN   
2      6.0   2.0   4.0   NaN   NaN   NaN   1.0   1.0   1.0   NaN  ...    NaN   
3      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
4      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    NaN   
5      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...    1.0   
6      NaN   NaN   NaN   NaN   1.0   NaN   NaN   NaN   NaN   NaN  ...    NaN   

                                                            
item 50003 50004 50005 50008 50009 50016 50017 50018 50028  
user                                                        
1      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
2      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
3      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
4      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
5      3.0   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  
6      1.0   2.0   2.0   1.0   1.0   1.0   1.0   1.0   1.0  

[6 rows x 62 columns]

## from collections import defaultdict
from surprise import SVD
from surprise import Reader, Dataset
import pandas as pd
from surprise import dump
from surprise import accuracy

def get_top_n(predictions, n=10):
    
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

train_data= pd.read_csv(r"./testdata.csv")
#loading into dataframe
df=pd.DataFrame(train_data)
df.head()

#reader class using only the rating parameter
reader= Reader(rating_scale=(-5.0, 5.0))

#arranging dataframe
data= Dataset.load_from_df(df[['user', 'item', 'rate']], reader)


#build full trainset and use cross validation for evaluation
trainset= data.build_full_trainset()

#pip install --upgrade tf-nightly-gpu-2.0-preview tfp-nightly


bsl_options = {'method': 'sgd',
               'learning_rate': .00005,
               }

algo = SVD()

algo.fit(trainset)


# Than predict ratings for all pairs (u, i) that are NOT in the training set.
testset = trainset.build_anti_testset()
predictions = algo.test(testset)
accuracy.rmse(predictions)
top_n = get_top_n(predictions, n=5)

# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

#1 [20106, 20052, 20123]    
#20106 id3 rate 4
#20106 id4 rate 5

#20052 id5 rate 1
#20052 id3 rate 6
#20052 id4 rate 5

#20123 id4 rate4
